## Data test train source split

In [39]:
# storing file names
import numpy as np 
import pandas as pd 

#src_class = input('BH/NS : ')
src_class = "NS"
frac = 1

data = pd.read_csv('source_list/'+src_class+'_data_clean_source_list.csv')
data =  data.sample(frac=1)
frac_no = int(len(data)*frac)
train_data = data.iloc[:frac_no]
test_data = data.iloc[frac_no:]
train_data.to_csv('source_list/final_list/'+src_class+'_train.csv')

import os
ctg = src_class

def hr():
    print('----------------------------------------------------------------------------')

# Copy sources data to respective folders

data = pd.read_csv('source_list/final_list/'+ctg+'_train.csv')
ch_names = [di.replace(' ' ,'_')[:-1] for di in data['B_NAME']]
a_names = [di.replace(' ' ,'_')[:-1]+'.csv' for di in data['A_NAME']]
src_id = [si[:6] for si in data['SRC_ID']]
#print(src_id)
os.system('rm -r '+ctg+'_data/train/')
os.system('mkdir '+ctg+'_data/train/')
for c,s ,a in zip(ch_names , src_id , a_names):
    try:
        src = ctg+'_data_all/'+c+'.csv' 
        ## adding source_id in file names
        dest = ctg+'_data/train/'+ s+'-'+c+'#'+a 
        #print('Copying' , c )
        os.system('cp '+src+' '+dest)
    except:
        hr()
        print('check source : ' , c )
        hr()

hr()
print('TRAINING DATA STORED')

----------------------------------------------------------------------------
TRAINING DATA STORED


## Append all data in one file

In [40]:
from ipywidgets import IntProgress
from IPython.display import display

src_class = src_class

def hr():
    print('_______________________________________________________________________________________')


def to_float(d):
    temp = []
    for di in d:
        
        try:
            temp.append(float(di))
        except Exception as e:
            print(e)
            print('NaN detected resetting value')
            temp.append(np.NAN)
    temp = np.asarray(temp)
    return temp 



In [41]:
set_type = 'train'
os.system('ls '+src_class+'_data/'+set_type+'/ > file_list')
file_list = pd.read_csv('file_list' , names=['names'])['names']
data = pd.DataFrame()

for file in file_list:
    data_temp =  pd.read_csv(src_class+'_data/'+set_type+'/'+file)
    ## extracting source_id from file names 
    src_id_name = file[:6]
    src_id = [src_id_name]*len(data_temp)
    src_id = np.asarray(src_id)
    data_temp.insert(0 ,'src_id' , src_id)
    src_n_name = file[29:-4].replace('_' , ' ')
    src_n = [src_n_name]*len(data_temp)
    src_n = np.asarray(src_n)
    data_temp.insert(0 ,'src_n' , src_n)
    data =  data.append(data_temp)
data = data.replace('      NaN' , np.nan)
data.to_csv('raw_data/'+src_class+'_'+set_type+'.csv')
data.describe()

hr()

# Data Processing

## Filtering

In [ ]:
flag_params = ['streak_src_flag' , 'sat_src_flag' , 'pileup_flag' , 'mstr_sat_src_flag' , ]


In [ ]:
def correct_bool(arr):
    temp = []
    #print(arr)
    for a in arr:
        #print(a , type(a))
        if(type(a) == bool):
            temp.append(a)
        else:
            if(a==' TRUE' or a=='True' or a=='  NAN'):
                temp.append(True)
            elif(a=='FALSE' or a=='False'):
                temp.append(False)
            else:
                print('something wrong' , a , type(a))
    return temp

In [ ]:
def to_float(d):
    temp = []
    for di in d:
        
        try:
            temp.append(float(di))
        except Exception as e:
            print(e)
            print('NaN detected resetting value')
            temp.append(np.NAN)
    temp = np.asarray(temp)
    return temp 

In [ ]:
data = pd.read_csv('raw_data/'+src_class+'_'+set_type+'.csv')
print('Data  Before flag filtering')
display(data.describe()[:1])
for f in flag_params:
    temp_flag = data[f]
    temp_flag = correct_bool(temp_flag)
    data[f] = temp_flag
    #print(temp_flag)
    data = data[data[f]==False]

#display(data)
data = data.dropna(how='all' , axis=1)
data = data.dropna(how='all' , axis=0)
display(data.describe()[:1])

Data  Before flag filtering


,Unnamed: 0,index,sepn,ra,dec,obsid,obi,src_rate_aper_m,src_rate_aper_lolim_m,src_rate_aper_hilim_m,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
count,227.0,227.0,227.0,227.0,227.0,227.0,227.0,216.0,216.0,216.0,...,211.0,201.0,201.0,201.0,227.0,227.0,227.0,227.0,227.0,227.0


,Unnamed: 0,index,sepn,ra,dec,obsid,obi,src_rate_aper_m,src_rate_aper_lolim_m,src_rate_aper_hilim_m,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
count,199.0,199.0,199.0,199.0,199.0,199.0,199.0,189.0,189.0,189.0,...,185.0,175.0,175.0,175.0,199.0,199.0,199.0,199.0,199.0,199.0


take average of Band params

In [ ]:
band_params = [ 'photflux_aper' ,'photflux_aper_lolim' , 'photflux_aper_hilim' , 'flux_aper' , 'flux_aper_lolim' , 'flux_aper_hilim' , 'var_index' ,'var_prob' , 'ks_prob' , 'kp_prob' ,'var_sigma' ,'var_mean' , 'var_min' ,  'var_max' , 'var_inter_index' , 'var_inter_prob' , 'var_inter_sigma' , 'flux_significance']

filters = ['_b' ,'_h' ,'_m' ,'_u' ,'_s']

combining band parameterd

In [ ]:
data_band_mean = pd.DataFrame()
data_band_mean.index.name = 'name'
for param in band_params[:]:
    temp_row = [param+f for f in filters]
    data_temp = data[temp_row][:]

    data_temp_new = pd.DataFrame()
    for j in data_temp:
        data_temp_new.insert( 0 ,j ,to_float(data_temp[j].to_numpy()))
    temp_mean =  data_temp_new.mean(axis=1)
    data_band_mean.insert(0 , param , temp_mean)

display(data_band_mean)
#data_band_mean.to_csv('temp.csv')

,flux_significance,var_inter_sigma,var_inter_prob,var_inter_index,var_max,var_min,var_mean,var_sigma,kp_prob,ks_prob,var_prob,var_index,flux_aper_hilim,flux_aper_lolim,flux_aper,photflux_aper_hilim,photflux_aper_lolim,photflux_aper
name,,,,,,,,,,,,,,,,,,
0,6.106,1.183580e-05,0.99640,7.80,0.002537,0.001183,0.001257,0.000121,0.4978,0.5988,0.4218,1.0,7.901940e-15,5.858260e-15,6.887600e-15,3.534620e-06,2.672100e-06,3.107660e-06
1,8.824,1.183580e-05,0.99640,7.80,0.003358,0.003261,0.003306,0.000035,0.4264,0.4672,0.1838,0.0,2.102280e-14,1.723976e-14,1.917356e-14,9.405200e-06,7.602000e-06,8.517000e-06
2,9.848,1.183580e-05,0.99640,7.80,0.007894,0.004829,0.005101,0.000426,0.4292,0.5602,0.3546,0.4,4.579740e-14,3.861600e-14,4.205200e-14,1.854200e-05,1.403600e-05,1.629020e-05
3,158.520,3.960122e-03,0.99280,7.20,2.903000,2.891000,2.897000,0.006242,0.9420,0.9870,0.1540,0.0,9.644000e-11,9.524000e-11,9.584000e-11,8.574000e-03,8.467000e-03,8.520000e-03
4,166.390,3.960122e-03,0.99280,7.20,3.219000,3.219000,3.219000,0.000268,0.9650,0.9740,0.0130,0.0,1.495000e-10,1.477000e-10,1.486000e-10,1.362000e-02,1.346000e-02,1.354000e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0.944,1.352975e-07,0.39325,1.75,0.000199,0.000136,0.000137,0.000006,0.4170,0.5670,0.4835,0.0,5.374500e-16,3.662500e-17,8.847500e-17,4.444600e-07,6.314000e-08,1.582200e-07
195,0.490,1.352975e-07,0.39325,1.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.533000e-16,5.435000e-17,2.355500e-16,6.539600e-07,1.656200e-08,9.718000e-08
196,0.448,1.352975e-07,0.39325,1.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.483750e-16,0.000000e+00,0.000000e+00,8.029800e-07,1.245600e-08,1.083200e-07


Rescaling flux parameters

In [ ]:
flux_params = ['photflux_aper' ,'photflux_aper_lolim' , 'photflux_aper_hilim' , 'flux_aper' , 'flux_aper_lolim' , 'flux_aper_hilim' ]
flux_params_model = ['flux_bb' , 'flux_bb_lolim' , 'flux_bb_hilim' , 'flux_brems' , 'flux_brems_hilim' , 'flux_brems_lolim' ,  'flux_powlaw' , 'flux_powlaw_lolim' , 'flux_powlaw_hilim' , ]
flux_params_band = []
for fp in flux_params:
    for f in filters:
        temp = fp+f
        flux_params_band.append(temp)
flux_params_extend = flux_params_band+flux_params

In [ ]:
data_band_flux = data[flux_params_band] 
data_model_flux = data[flux_params_model].reset_index(drop=True)
data_band_flux = data_band_flux.reset_index(drop=True)
data_band_flux = data_band_flux.replace(0.0 , np.NAN)

display((data_band_flux))
data_mean_flux = data_band_mean[flux_params].reset_index(drop=True)
display(data_mean_flux)
data_flux = pd.concat([data_model_flux , data_mean_flux , data_band_flux] , axis=1)
data_flux = data_flux.replace(0 , np.nan)
data_flux = data_flux.replace('0' , np.nan)
data_flux = data_flux.replace('0.0' , np.nan)
display(data_flux)
data_flux.to_csv('temp.csv')

,photflux_aper_b,photflux_aper_h,photflux_aper_m,photflux_aper_u,photflux_aper_s,photflux_aper_lolim_b,photflux_aper_lolim_h,photflux_aper_lolim_m,photflux_aper_lolim_u,photflux_aper_lolim_s,...,flux_aper_lolim_b,flux_aper_lolim_h,flux_aper_lolim_m,flux_aper_lolim_u,flux_aper_lolim_s,flux_aper_hilim_b,flux_aper_hilim_h,flux_aper_hilim_m,flux_aper_hilim_u,flux_aper_hilim_s
0,8.939000e-06,1.852000e-06,1.534000e-06,5.733000e-07,2.640000e-06,8.083000e-06,1.466000e-06,1.269000e-06,2.565000e-07,2.286000e-06,...,1.529000e-14,7.312000e-15,3.163000e-15,1.653000e-16,3.361000e-15,1.871000e-14,1.136000e-14,4.488000e-15,5.327000e-16,4.419000e-15
1,2.350000e-05,4.127000e-06,5.416000e-06,2.091000e-06,7.451000e-06,2.189000e-05,3.484000e-06,4.866000e-06,1.019000e-06,6.751000e-06,...,4.418000e-14,2.004000e-14,1.112000e-14,4.288000e-16,1.043000e-14,5.045000e-14,2.697000e-14,1.364000e-14,1.444000e-15,1.261000e-14
2,3.845000e-05,1.015000e-05,9.311000e-06,1.143000e-05,1.211000e-05,3.620000e-05,9.140000e-06,8.422000e-06,5.568000e-06,1.085000e-05,...,9.878000e-14,5.518000e-14,2.061000e-14,NaN,1.851000e-14,1.108000e-13,6.842000e-14,2.500000e-14,1.967000e-15,2.280000e-14
3,NaN,8.520000e-03,NaN,NaN,NaN,NaN,8.467000e-03,NaN,NaN,NaN,...,NaN,9.524000e-11,NaN,NaN,NaN,NaN,9.644000e-11,NaN,NaN,NaN
4,NaN,1.354000e-02,NaN,NaN,NaN,NaN,1.346000e-02,NaN,NaN,NaN,...,NaN,1.477000e-10,NaN,NaN,NaN,NaN,1.495000e-10,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,4.438000e-07,9.860000e-08,NaN,NaN,2.487000e-07,2.102000e-07,NaN,NaN,NaN,1.055000e-07,...,NaN,NaN,NaN,NaN,1.465000e-16,6.129000e-16,8.013000e-16,1.742000e-16,NaN,5.614000e-16
195,2.922000e-07,NaN,1.937000e-07,NaN,NaN,3.812000e-08,NaN,4.469000e-08,NaN,NaN,...,NaN,NaN,1.087000e-16,NaN,NaN,4.731000e-16,NaN,8.335000e-16,NaN,NaN
196,3.737000e-07,NaN,NaN,NaN,1.679000e-07,6.228000e-08,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.070000e-15,1.622000e-15,3.612000e-16,NaN,3.403000e-16
197,1.010000e-06,7.484000e-07,NaN,NaN,NaN,4.457000e-07,3.421000e-07,NaN,NaN,NaN,...,4.000000e-15,4.323000e-15,NaN,NaN,NaN,1.350000e-14,1.373000e-14,NaN,NaN,NaN


,photflux_aper,photflux_aper_lolim,photflux_aper_hilim,flux_aper,flux_aper_lolim,flux_aper_hilim
0,3.107660e-06,2.672100e-06,3.534620e-06,6.887600e-15,5.858260e-15,7.901940e-15
1,8.517000e-06,7.602000e-06,9.405200e-06,1.917356e-14,1.723976e-14,2.102280e-14
2,1.629020e-05,1.403600e-05,1.854200e-05,4.205200e-14,3.861600e-14,4.579740e-14
3,8.520000e-03,8.467000e-03,8.574000e-03,9.584000e-11,9.524000e-11,9.644000e-11
4,1.354000e-02,1.346000e-02,1.362000e-02,1.486000e-10,1.477000e-10,1.495000e-10
...,...,...,...,...,...,...
194,1.582200e-07,6.314000e-08,4.444600e-07,8.847500e-17,3.662500e-17,5.374500e-16
195,9.718000e-08,1.656200e-08,6.539600e-07,2.355500e-16,5.435000e-17,6.533000e-16
196,1.083200e-07,1.245600e-08,8.029800e-07,0.000000e+00,0.000000e+00,8.483750e-16
197,3.516800e-07,1.575600e-07,4.963780e-06,8.952000e-15,4.161500e-15,1.361500e-14


,flux_bb,flux_bb_lolim,flux_bb_hilim,flux_brems,flux_brems_hilim,flux_brems_lolim,flux_powlaw,flux_powlaw_lolim,flux_powlaw_hilim,photflux_aper,...,flux_aper_lolim_b,flux_aper_lolim_h,flux_aper_lolim_m,flux_aper_lolim_u,flux_aper_lolim_s,flux_aper_hilim_b,flux_aper_hilim_h,flux_aper_hilim_m,flux_aper_hilim_u,flux_aper_hilim_s
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.107660e-06,...,1.529000e-14,7.312000e-15,3.163000e-15,1.653000e-16,3.361000e-15,1.871000e-14,1.136000e-14,4.488000e-15,5.327000e-16,4.419000e-15
1,3.242000e-14,2.902000e-14,3.542000e-14,4.118000e-14,4.432000e-14,3.683000e-14,4.273000e-14,3.918000e-14,4.663000e-14,8.517000e-06,...,4.418000e-14,2.004000e-14,1.112000e-14,4.288000e-16,1.043000e-14,5.045000e-14,2.697000e-14,1.364000e-14,1.444000e-15,1.261000e-14
2,7.683000e-14,7.126000e-14,8.323000e-14,9.033000e-14,9.683000e-14,8.257000e-14,9.316000e-14,8.634000e-14,9.941000e-14,1.629020e-05,...,9.878000e-14,5.518000e-14,2.061000e-14,NaN,1.851000e-14,1.108000e-13,6.842000e-14,2.500000e-14,1.967000e-15,2.280000e-14
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.520000e-03,...,NaN,9.524000e-11,NaN,NaN,NaN,NaN,9.644000e-11,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.354000e-02,...,NaN,1.477000e-10,NaN,NaN,NaN,NaN,1.495000e-10,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.582200e-07,...,NaN,NaN,NaN,NaN,1.465000e-16,6.129000e-16,8.013000e-16,1.742000e-16,NaN,5.614000e-16
195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.718000e-08,...,NaN,NaN,1.087000e-16,NaN,NaN,4.731000e-16,NaN,8.335000e-16,NaN,NaN
196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.083200e-07,...,NaN,NaN,NaN,NaN,NaN,1.070000e-15,1.622000e-15,3.612000e-16,NaN,3.403000e-16
197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.516800e-07,...,4.000000e-15,4.323000e-15,NaN,NaN,NaN,1.350000e-14,1.373000e-14,NaN,NaN,NaN


In [ ]:
data_flux_log = pd.DataFrame()
for f in data_flux:
    temp = data_flux[f]
    temp = np.log10(temp)
#    display(temp)
    data_flux_log.insert(0 , f , temp)

display(data_flux_log)

,flux_aper_hilim_s,flux_aper_hilim_u,flux_aper_hilim_m,flux_aper_hilim_h,flux_aper_hilim_b,flux_aper_lolim_s,flux_aper_lolim_u,flux_aper_lolim_m,flux_aper_lolim_h,flux_aper_lolim_b,...,photflux_aper,flux_powlaw_hilim,flux_powlaw_lolim,flux_powlaw,flux_brems_lolim,flux_brems_hilim,flux_brems,flux_bb_hilim,flux_bb_lolim,flux_bb
0,-14.354676,-15.273517,-14.347947,-13.944622,-13.727926,-14.473531,-15.781727,-14.499901,-14.135964,-13.815593,...,-5.507567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-13.899285,-14.840433,-13.865186,-13.569119,-13.297139,-13.981716,-15.367745,-13.953895,-13.698102,-13.354774,...,-5.069713,-13.331335,-13.406936,-13.369267,-13.433798,-13.35340,-13.385314,-13.450751,-13.537303,-13.489187
2,-13.642065,-14.706196,-13.602060,-13.164817,-12.955460,-13.732594,NaN,-13.685922,-13.258218,-13.005331,...,-4.788074,-13.002570,-13.063788,-13.030771,-13.083178,-13.01399,-13.044168,-13.079720,-13.147154,-13.114469
3,NaN,NaN,NaN,-10.015743,NaN,NaN,NaN,NaN,-10.021181,NaN,...,-2.069560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,-9.825359,NaN,NaN,NaN,NaN,-9.830620,NaN,...,-1.868381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,-15.250728,NaN,-15.758952,-15.096205,-15.212610,-15.834162,NaN,NaN,NaN,NaN,...,-6.800739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,NaN,NaN,-15.079094,NaN,-15.325047,NaN,NaN,-15.963770,NaN,NaN,...,-7.012423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,-15.468138,NaN,-15.442252,-14.789949,-14.970616,NaN,NaN,NaN,NaN,NaN,...,-6.965291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,NaN,NaN,NaN,-13.862329,-13.869666,NaN,NaN,NaN,-14.364215,-14.397940,...,-6.453852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
var_params = [ 'var_index' ,'var_prob' , 'ks_prob' , 'kp_prob' ,'var_sigma' ,'var_mean' , 'var_min' ,  'var_max' , 'var_inter_index' , 'var_inter_prob' , 'var_inter_sigma']
data_var = data_band_mean[var_params].reset_index(drop=True)
display(data_var)

,var_index,var_prob,ks_prob,kp_prob,var_sigma,var_mean,var_min,var_max,var_inter_index,var_inter_prob,var_inter_sigma
0,1.0,0.4218,0.5988,0.4978,0.000121,0.001257,0.001183,0.002537,7.80,0.99640,1.183580e-05
1,0.0,0.1838,0.4672,0.4264,0.000035,0.003306,0.003261,0.003358,7.80,0.99640,1.183580e-05
2,0.4,0.3546,0.5602,0.4292,0.000426,0.005101,0.004829,0.007894,7.80,0.99640,1.183580e-05
3,0.0,0.1540,0.9870,0.9420,0.006242,2.897000,2.891000,2.903000,7.20,0.99280,3.960122e-03
4,0.0,0.0130,0.9740,0.9650,0.000268,3.219000,3.219000,3.219000,7.20,0.99280,3.960122e-03
...,...,...,...,...,...,...,...,...,...,...,...
194,0.0,0.4835,0.5670,0.4170,0.000006,0.000137,0.000136,0.000199,1.75,0.39325,1.352975e-07
195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.75,0.39325,1.352975e-07
196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.75,0.39325,1.352975e-07
197,0.0,0.4760,0.2260,0.1860,0.000015,0.000256,0.000251,0.000367,6.00,0.92500,2.876000e-07


In [ ]:
single_params = [ 'flux_powlaw'  , 'powlaw_gamma' , 'powlaw_nh' ,  'powlaw_ampl' ,  'powlaw_stat' , 'bb_kt' , 'bb_nh' ,  'bb_ampl' ,  'bb_stat'  , 'brems_kt' , 'brems_nh' ,'brems_stat' ,]

hard_params = ['hard_hm' ,'hard_hm_hilim' ,'hard_hm_lolim' ,  'hard_hs' , 'hard_ms' ,'hard_hs_hilim' ,'hard_hs_lolim' ,'hard_ms_hilim' ,'hard_ms_lolim',]

model_fit_params = [ 'powlaw_gamma' , 'powlaw_gamma_hilim' , 'powlaw_gamma_lolim' ,'powlaw_nh' , 'powlaw_nh_hilim' , 'powlaw_nh_lolim' ,  'powlaw_ampl' ,  'powlaw_ampl_hilim' , 'powlaw_ampl_lolim' , 'powlaw_stat' , 'bb_kt' ,'bb_kt_hilim' , 'bb_kt_lolim' , 'bb_nh_hilim' , 'bb_nh_lolim' , 'bb_nh' ,  'bb_ampl' ,'bb_ampl_lolim' ,'bb_ampl_hilim' ,   'bb_stat' ,  'brems_kt' , 'brems_kt_hilim' ,'brems_kt_lolim' , 'brems_nh' , 'brems_nh_hilim' ,'brems_nh_lolim' ,'brems_stat' ,]

obs_info_params = [ 'livetime','significance','likelihood','pileup_flag','mstr_sat_src_flag','mstr_streak_src_flag'   ,'gti_obs' , 'flux_significance_b'  , 'flux_significance_m' , 'flux_significance_s' , 'flux_significance_h' , 'flux_significance_u'    ]

single_params_lim = [ ]

In [ ]:
data_rest = data[hard_params+model_fit_params].reset_index(drop=True)
display(data_rest)


,hard_hm,hard_hm_hilim,hard_hm_lolim,hard_hs,hard_ms,hard_hs_hilim,hard_hs_lolim,hard_ms_hilim,hard_ms_lolim,powlaw_gamma,...,bb_ampl_lolim,bb_ampl_hilim,bb_stat,brems_kt,brems_kt_hilim,brems_kt_lolim,brems_nh,brems_nh_hilim,brems_nh_lolim,brems_stat
0,0.0974,0.2255,-0.0381,-0.1761,-0.2673,-0.0606,-0.2954,-0.1655,-0.3679,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.1324,-0.0468,-0.2217,-0.2898,-0.1624,-0.2105,-0.3691,-0.0956,-0.2280,2.451,...,0.000068,0.000093,1.936,2.481,3.134,2.000,11.05,15.48,6.992,0.859
2,0.0400,0.1068,-0.0281,-0.0912,-0.1299,-0.0194,-0.1618,-0.0618,-0.1968,2.193,...,0.000060,0.000078,1.743,3.440,4.313,2.799,13.14,17.89,8.830,0.814
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0.9994,1.0000,0.0981,-0.4160,-0.9994,0.0119,-0.7701,-0.4391,-1.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,-0.9994,-0.2392,-1.0000,-0.9994,0.9994,0.3117,-1.0000,1.0000,0.1593,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,0.5147,1.0000,-0.1480,0.0849,-0.3748,0.5228,-0.6402,0.0506,-1.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,0.9994,1.0000,0.7139,0.9994,-0.9994,1.0000,0.4828,0.0356,-1.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data_obs_info = data[obs_info_params].reset_index(drop=True)
display(data_obs_info)

,livetime,significance,likelihood,pileup_flag,mstr_sat_src_flag,mstr_streak_src_flag,gti_obs,flux_significance_b,flux_significance_m,flux_significance_s,flux_significance_h,flux_significance_u
0,41932.7,23.63,3822.52900,False,False,False,2000-02-29T04:03:25,10.45,5.80,7.46,4.97,1.85
1,39623.0,23.63,3822.52900,False,False,False,2005-08-20T09:03:34,15.08,9.85,10.64,6.60,1.95
2,29673.9,23.63,3822.52900,False,False,False,2013-12-09T01:11:41,17.13,10.47,9.63,10.06,1.95
3,11611.1,166.94,151803.00000,False,False,False,2001-07-21T04:45:23,NaN,NaN,NaN,158.52,NaN
4,11611.1,166.94,151803.00000,False,False,False,2001-07-21T11:42:23,NaN,NaN,NaN,166.39,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
194,39516.5,2.44,20.26300,False,False,False,2010-09-30T23:42:24,1.95,0.00,1.74,1.03,0.00
195,19807.2,2.44,20.26300,False,False,False,2012-10-31T17:20:25,1.15,1.30,0.00,0.00,0.00
196,19803.6,2.44,20.26300,False,False,False,2012-11-03T09:23:02,1.24,0.00,1.00,0.00,0.00
197,19260.2,1.84,17.85079,False,False,False,2011-06-16T12:41:48,1.84,0.00,0.00,1.84,0.00


In [ ]:
data_identify = data[['src_n' , 'src_id']].reset_index(drop=True)
print(data_identify)

                    src_n  src_id
0    1A 0620-00            BH0001
1    1A 0620-00            BH0001
2    1A 0620-00            BH0001
3    1E 1740.7-2942        BH0002
4    1E 1740.7-2942        BH0002
..                    ...     ...
194  GS 1354-645           BH0031
195  GS 1354-645           BH0031
196  GS 1354-645           BH0031
197  4U1630-472            BH0032
198  CJ0422+32             BH0033

[199 rows x 2 columns]


In [ ]:
data_combined = pd.concat([data_identify , data_obs_info , data_flux_log , data_var , data_rest] , axis=1)
obs_col = [src_class+'_OBS_'+str(i) for i in range(len(data_combined))]
data_combined.insert(0 , 'obs_id' , obs_col)
data_combined.insert( 0 , 'class' , [src_class]*len(data_combined))
data_combined = data_combined.reset_index(drop=True)
data_combined.index.name = 'index'
display(data_combined)
data_combined.to_csv('../processed_data/'+src_class+'_.csv')

,class,obs_id,src_n,src_id,livetime,significance,likelihood,pileup_flag,mstr_sat_src_flag,mstr_streak_src_flag,...,bb_ampl_lolim,bb_ampl_hilim,bb_stat,brems_kt,brems_kt_hilim,brems_kt_lolim,brems_nh,brems_nh_hilim,brems_nh_lolim,brems_stat
index,,,,,,,,,,,,,,,,,,,,,
0,BH,BH_OBS_0,1A 0620-00,BH0001,41932.7,23.63,3822.52900,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BH,BH_OBS_1,1A 0620-00,BH0001,39623.0,23.63,3822.52900,False,False,False,...,0.000068,0.000093,1.936,2.481,3.134,2.000,11.05,15.48,6.992,0.859
2,BH,BH_OBS_2,1A 0620-00,BH0001,29673.9,23.63,3822.52900,False,False,False,...,0.000060,0.000078,1.743,3.440,4.313,2.799,13.14,17.89,8.830,0.814
3,BH,BH_OBS_3,1E 1740.7-2942,BH0002,11611.1,166.94,151803.00000,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BH,BH_OBS_4,1E 1740.7-2942,BH0002,11611.1,166.94,151803.00000,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,BH,BH_OBS_194,GS 1354-645,BH0031,39516.5,2.44,20.26300,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,BH,BH_OBS_195,GS 1354-645,BH0031,19807.2,2.44,20.26300,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,BH,BH_OBS_196,GS 1354-645,BH0031,19803.6,2.44,20.26300,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
